In [1]:
!pip install bs4
!pip install requests
!pip install pandas

# Extract Links

In [29]:
from bs4 import BeautifulSoup
import requests
import pandas as pd



PAGENUMBER = 2
url = "https://www.namasha.com/Anime4i"
all_links = []

for i in range(PAGENUMBER):
    res = requests.get(url)
    soup = BeautifulSoup(res.content, 'html.parser')

    video_ids = [i.get("data-id") for i in soup.find_all("div", attrs={"class": "thumbnail"})]
    video_links = [i.find("a").get("href") for i in soup.find_all("div", attrs={"class": "d-flex align-items-start mb-n2"})]
    all_links.append(video_links)
    url = f"{url}?page={i}&qm=&lastId={video_ids[-1]}"


urls = []
for i in all_links:
    for j in i:
        urls.append(j)
print(len(urls))

1200


In [31]:
urls

['https://www.namasha.com/v/VqWVtIBI',
 'https://www.namasha.com/v/iedmatMR',
 'https://www.namasha.com/v/ZYVHfIkc',
 'https://www.namasha.com/v/dzpjgkeW',
 'https://www.namasha.com/v/XkxqyLJS',
 'https://www.namasha.com/v/JaUKBqo9',
 'https://www.namasha.com/v/Qp7SlZZk',
 'https://www.namasha.com/v/D8T7PaLd',
 'https://www.namasha.com/v/8TFpBZTn',
 'https://www.namasha.com/v/agYwBicR',
 'https://www.namasha.com/v/jmxViVNO',
 'https://www.namasha.com/v/Z4xnDsO9',
 'https://www.namasha.com/v/QG5Kjciy',
 'https://www.namasha.com/v/YM7blwUM',
 'https://www.namasha.com/v/yJf165tO',
 'https://www.namasha.com/v/uX765dZ7',
 'https://www.namasha.com/v/CzZDdHEN',
 'https://www.namasha.com/v/6fzPc4iM',
 'https://www.namasha.com/v/UvpwsjtR',
 'https://www.namasha.com/v/eHrgsW55',
 'https://www.namasha.com/v/xsBv5eGe',
 'https://www.namasha.com/v/CX3Ny1RG',
 'https://www.namasha.com/v/rgTLhp8T',
 'https://www.namasha.com/v/sJLNnLSW',
 'https://www.namasha.com/v/zUSNXqAJ',
 'https://www.namasha.com

# Get Information of links

In [30]:
df = pd.DataFrame()
urls = urls[:65]

for url in urls:

    # url = "https://www.namasha.com/v/uRrRaQ7F"
    res = requests.get(url)
    soup = BeautifulSoup(res.content, 'html.parser')

    title = soup.find("h1", attrs={"class": "video-title font-size-base font-size-lg-lg font-weight-bold mb-1"}).text.strip()
    link  = soup.find("span", attrs={"class": "video-like-count"}).text
    count_comment = soup.find("h4", attrs={"class": "font-size-base font-size-lg-sm mb-0"}).text.strip()
    time = soup.find("time" , attrs={"class", "text-dynamic-half-dark font-size-xs font-weight-light font-weight-lg-medium mt-2"}).text.strip()
    
    data = {
        'titles': [title],
        'likes': [link],
        "comments": [count_comment],
        "times":[time],
        "urls" :url}
    

    df2 = pd.DataFrame(data)
    df = pd.concat([df, df2], ignore_index=True)

    
df.to_csv(f"namasha_channel{len(urls)}.csv")

In [39]:
df.likes = df.likes.apply(lambda x: int(x))

# cleaning dataset

In [42]:
df

,titles,likes,comments,times,urls
0,انیمه شکارچی شیطان the demon hunter فصل 2 قسمت...,32,نظرات,منتشر شده در تاریخ ۱۴۰۳/۰۴/۲۳,https://www.namasha.com/v/VqWVtIBI
1,انیمه سلسله جید Jade Dynasty فصل 2 قسمت 18 (44...,48,نظرات,منتشر شده در تاریخ ۱۴۰۳/۰۴/۲۲,https://www.namasha.com/v/iedmatMR
2,انیمه سرزمین ارواح : فرقه تانگ بی رقیب Soul La...,81,نظرات,منتشر شده در تاریخ ۱۴۰۳/۰۴/۲۲,https://www.namasha.com/v/ZYVHfIkc
3,انیمه جاودانگی Immortality فصل 3 قسمت 15 زیرنو...,46,نظرات,منتشر شده در تاریخ ۱۴۰۳/۰۴/۲۲,https://www.namasha.com/v/dzpjgkeW
4,انیمه تصفیه صد مانترای اعظم Hundread Refining ...,7,نظرات,منتشر شده در تاریخ ۱۴۰۳/۰۴/۲۲,https://www.namasha.com/v/XkxqyLJS
...,...,...,...,...,...
60,انیمه تصفیه صد مانترای اعظم Hundread Refining ...,11,نظرات,منتشر شده در تاریخ ۱۴۰۳/۰۴/۱۷,https://www.namasha.com/v/1skisi8J
61,انیمه سراشپز جادویی یخ و اتش Magic Chef of Ice...,16,نظرات,منتشر شده در تاریخ ۱۴۰۳/۰۴/۱۷,https://www.namasha.com/v/Ru4cZVRT
62,انیمه هزار جهان (Wan Jie Du Zun) قسمت 249 زیرن...,37,نظرات,منتشر شده در تاریخ ۱۴۰۳/۰۴/۱۷,https://www.namasha.com/v/44gmpVjZ
63,انیمه 100000 تسویه چی (One Hundred Thousand Ye...,25,نظرات,منتشر شده در تاریخ ۱۴۰۳/۰۴/۱۷,https://www.namasha.com/v/fzuJ5dAC


In [45]:
df.sort_values("likes", ascending=False)

,titles,likes,comments,times,urls
55,انیمه جاودانه شورشی XIAN NI (Renegade Immortal...,109,نظرات,منتشر شده در تاریخ ۱۴۰۳/۰۴/۱۷,https://www.namasha.com/v/3K52GfTU
2,انیمه سرزمین ارواح : فرقه تانگ بی رقیب Soul La...,81,نظرات,منتشر شده در تاریخ ۱۴۰۳/۰۴/۲۲,https://www.namasha.com/v/ZYVHfIkc
16,انیمه (Perfect World (Wanmei Shijie قسمت 171 ز...,75,نظرات,منتشر شده در تاریخ ۱۴۰۳/۰۴/۲۲,https://www.namasha.com/v/CzZDdHEN
54,انیمه ستاره بلعیده شده Swallowed Star قسمت 127...,66,نظرات,منتشر شده در تاریخ ۱۴۰۳/۰۴/۱۸,https://www.namasha.com/v/gOLlq6Ux
59,انیمه پادشاه هنرهای زرمی legend of martial imm...,65,نظرات,منتشر شده در تاریخ ۱۴۰۳/۰۴/۱۷,https://www.namasha.com/v/cK6zyuNq
...,...,...,...,...,...
33,انیمه تصفیه صد مانترای اعظم Hundread Refining ...,8,نظرات,منتشر شده در تاریخ ۱۴۰۳/۰۴/۲۱,https://www.namasha.com/v/SxqZQMWs
4,انیمه تصفیه صد مانترای اعظم Hundread Refining ...,7,نظرات,منتشر شده در تاریخ ۱۴۰۳/۰۴/۲۲,https://www.namasha.com/v/XkxqyLJS
20,انیمه افسانه رودخانه تاریک فصل 2 Tales Of Dark...,7,نظرات,منتشر شده در تاریخ ۱۴۰۳/۰۴/۲۱,https://www.namasha.com/v/xsBv5eGe
6,انیمه نیروانا سوار طوفان Nirvana of Storm Ride...,5,نظرات,منتشر شده در تاریخ ۱۴۰۳/۰۴/۲۲,https://www.namasha.com/v/Qp7SlZZk


In [16]:
def clean_comments(x):
    rm_list = ["(" ,")", "نظرات"]

    for trash_words in rm_list:
        if trash_words in x:
            x = x.replace(trash_words , "")
    

    if x == "":
        return 0
    
    return int(x)
    


df["likes"] = df["likes"].apply(lambda x: int(x))
df["comments"] = df["comments"].apply(lambda x:clean_comments(x))

In [18]:
df.sort_values("likes", ascending=False).head(20)

,titles,likes,comments,times,urls
655,انیمه جاودانه شورشی XIAN NI (Renegade Immortal...,109,0,منتشر شده در تاریخ ۱۴۰۳/۰۴/۱۷,https://www.namasha.com/v/3K52GfTU
295,انیمه جاودانه شورشی XIAN NI (Renegade Immortal...,109,0,منتشر شده در تاریخ ۱۴۰۳/۰۴/۱۷,https://www.namasha.com/v/3K52GfTU
775,انیمه جاودانه شورشی XIAN NI (Renegade Immortal...,109,0,منتشر شده در تاریخ ۱۴۰۳/۰۴/۱۷,https://www.namasha.com/v/3K52GfTU
55,انیمه جاودانه شورشی XIAN NI (Renegade Immortal...,109,0,منتشر شده در تاریخ ۱۴۰۳/۰۴/۱۷,https://www.namasha.com/v/3K52GfTU
715,انیمه جاودانه شورشی XIAN NI (Renegade Immortal...,109,0,منتشر شده در تاریخ ۱۴۰۳/۰۴/۱۷,https://www.namasha.com/v/3K52GfTU
1015,انیمه جاودانه شورشی XIAN NI (Renegade Immortal...,109,0,منتشر شده در تاریخ ۱۴۰۳/۰۴/۱۷,https://www.namasha.com/v/3K52GfTU
835,انیمه جاودانه شورشی XIAN NI (Renegade Immortal...,109,0,منتشر شده در تاریخ ۱۴۰۳/۰۴/۱۷,https://www.namasha.com/v/3K52GfTU
175,انیمه جاودانه شورشی XIAN NI (Renegade Immortal...,109,0,منتشر شده در تاریخ ۱۴۰۳/۰۴/۱۷,https://www.namasha.com/v/3K52GfTU
415,انیمه جاودانه شورشی XIAN NI (Renegade Immortal...,109,0,منتشر شده در تاریخ ۱۴۰۳/۰۴/۱۷,https://www.namasha.com/v/3K52GfTU
235,انیمه جاودانه شورشی XIAN NI (Renegade Immortal...,109,0,منتشر شده در تاریخ ۱۴۰۳/۰۴/۱۷,https://www.namasha.com/v/3K52GfTU


In [ ]:
['https://www.namasha.com/v/VqWVtIBI',
 'https://www.namasha.com/v/iedmatMR',
 'https://www.namasha.com/v/ZYVHfIkc',
 'https://www.namasha.com/v/dzpjgkeW',
 'https://www.namasha.com/v/XkxqyLJS',
 'https://www.namasha.com/v/JaUKBqo9',
 'https://www.namasha.com/v/Qp7SlZZk',
 'https://www.namasha.com/v/D8T7PaLd',
 'https://www.namasha.com/v/8TFpBZTn',
 'https://www.namasha.com/v/agYwBicR',
 'https://www.namasha.com/v/jmxViVNO',
 'https://www.namasha.com/v/Z4xnDsO9',
 'https://www.namasha.com/v/QG5Kjciy',
 'https://www.namasha.com/v/YM7blwUM',
 'https://www.namasha.com/v/yJf165tO',
 'https://www.namasha.com/v/uX765dZ7',
 'https://www.namasha.com/v/CzZDdHEN',
 'https://www.namasha.com/v/6fzPc4iM',
 'https://www.namasha.com/v/UvpwsjtR',
 'https://www.namasha.com/v/eHrgsW55',
 'https://www.namasha.com/v/xsBv5eGe',
 'https://www.namasha.com/v/CX3Ny1RG',
 'https://www.namasha.com/v/rgTLhp8T',
 'https://www.namasha.com/v/sJLNnLSW',
 'https://www.namasha.com/v/zUSNXqAJ',
 'https://www.namasha.com/v/3dKCgqge',
 'https://www.namasha.com/v/LsGKIZj7',
 'https://www.namasha.com/v/dTKOeSWO',
 'https://www.namasha.com/v/jwdH9XnZ',
 'https://www.namasha.com/v/DfeF494s',
 'https://www.namasha.com/v/L9T97Lxy',
 'https://www.namasha.com/v/647hbmNJ',
 'https://www.namasha.com/v/shxRBgyJ',
 'https://www.namasha.com/v/SxqZQMWs',
 'https://www.namasha.com/v/MasrmS82',
 'https://www.namasha.com/v/GE6IM5SN',
 'https://www.namasha.com/v/8MColeyn',
 'https://www.namasha.com/v/qJdzhrkQ',
 'https://www.namasha.com/v/XqF3vhkn',
 'https://www.namasha.com/v/iEQkzk2p',
 'https://www.namasha.com/v/d521eRyK',
 'https://www.namasha.com/v/UVjhTXbK',
 'https://www.namasha.com/v/hNyvb3eb',
 'https://www.namasha.com/v/g7RvZZiI',
 'https://www.namasha.com/v/R7KKNno8',
 'https://www.namasha.com/v/Wn2e2dzt',
 'https://www.namasha.com/v/9hjfhyMk',
 'https://www.namasha.com/v/245lIH62',
 'https://www.namasha.com/v/oZESjQfP',
 'https://www.namasha.com/v/wGFIxivW',
 'https://www.namasha.com/v/kjMEgN4Q',
 'https://www.namasha.com/v/xjBHDSlu',
 'https://www.namasha.com/v/XIXFiS9R',
 'https://www.namasha.com/v/517tpaqX',
 'https://www.namasha.com/v/gOLlq6Ux',
 'https://www.namasha.com/v/3K52GfTU',
 'https://www.namasha.com/v/jAnE2IXv',
 'https://www.namasha.com/v/DJnJ1dJg',
 'https://www.namasha.com/v/LVx43YJ4',
 'https://www.namasha.com/v/cK6zyuNq',
 'https://www.namasha.com/v/1skisi8J',
 'https://www.namasha.com/v/Ru4cZVRT',
 'https://www.namasha.com/v/44gmpVjZ',
 'https://www.namasha.com/v/fzuJ5dAC',
 'https://www.namasha.com/v/i9TmxoM1']